In [69]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, \
  mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [70]:
df_returns = pd.DataFrame()

In [71]:
df_returns = pd.read_csv('IBRX-returns.csv', index_col=0, parse_dates=True)
df = pd.read_csv('IBRX100.csv', index_col=0, parse_dates=True)

#for name in df.columns:
#  df_returns[name] = np.log(df[name]).diff()

In [72]:
df.isna().sum().sum()

0

In [73]:
df_returns.head()

,VALE3.SA,PETR4.SA,PETR3.SA,ITUB4.SA,BBDC4.SA,B3SA3.SA,ABEV3.SA,JBSS3.SA,WEGE3.SA,SUZB3.SA,ITSA4.SA,BBAS3.SA,BRAX11.SA,BOVA11.SA
Date,,,,,,,,,,,,,,
2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-03,0.043537,0.030814,0.034304,0.024671,0.020945,0.017285,-0.011462,0.029801,0.023306,0.0,0.029193,-0.004539,0.021995,0.023738
2012-01-04,0.006765,0.007114,0.006225,0.001146,0.001593,0.017983,-0.023517,0.018820,0.005222,0.0,0.001742,-0.004975,0.000000,0.000000
2012-01-05,-0.027102,-0.011587,-0.006225,-0.007473,-0.006388,-0.010951,-0.019997,0.016807,0.006231,0.0,-0.004361,-0.010445,-0.009189,-0.013620
2012-01-06,-0.008637,0.000896,-0.000833,0.007473,0.012419,0.014903,-0.002528,0.011599,-0.011453,0.0,0.005231,-0.025522,-0.000250,0.002427


In [74]:
df_returns['BOVA11.SA'] = df_returns['BOVA11.SA'].shift(-1)
df_returns['BOVA11.SA'].tail()

Date
2022-01-19    0.008136
2022-01-20   -0.001908
2022-01-21   -0.005939
2022-01-24    0.021388
2022-01-25         NaN
Name: BOVA11.SA, dtype: float64

In [75]:
Ntest = 1000
train = df_returns.iloc[1:-Ntest]
test = df_returns.iloc[-Ntest:-1]

In [76]:
df_returns.columns

Index(['VALE3.SA', 'PETR4.SA', 'PETR3.SA', 'ITUB4.SA', 'BBDC4.SA', 'B3SA3.SA',
       'ABEV3.SA', 'JBSS3.SA', 'WEGE3.SA', 'SUZB3.SA', 'ITSA4.SA', 'BBAS3.SA',
       'BRAX11.SA', 'BOVA11.SA'],
      dtype='object')

In [77]:
# x_cols = df.columns.drop('SPY')
x_cols = ['VALE3.SA', 'PETR4.SA', 'PETR3.SA', 'ITUB4.SA', 'BBDC4.SA', 'B3SA3.SA',
          'ABEV3.SA', 'JBSS3.SA', 'WEGE3.SA', 'SUZB3.SA', 'ITSA4.SA', 'BBAS3.SA']
x_cols

['VALE3.SA',
 'PETR4.SA',
 'PETR3.SA',
 'ITUB4.SA',
 'BBDC4.SA',
 'B3SA3.SA',
 'ABEV3.SA',
 'JBSS3.SA',
 'WEGE3.SA',
 'SUZB3.SA',
 'ITSA4.SA',
 'BBAS3.SA']

In [78]:
Xtrain = train[x_cols]
Ytrain = train['BOVA11.SA']
Xtest = test[x_cols]
Ytest = test['BOVA11.SA']

In [79]:
Ytest.head()

Date
2018-01-10    0.013924
2018-01-11    0.000130
2018-01-12    0.005204
2018-01-15   -0.000130
2018-01-16    0.017367
Name: BOVA11.SA, dtype: float64

In [80]:
Xtrain.head()

,VALE3.SA,PETR4.SA,PETR3.SA,ITUB4.SA,BBDC4.SA,B3SA3.SA,ABEV3.SA,JBSS3.SA,WEGE3.SA,SUZB3.SA,ITSA4.SA,BBAS3.SA
Date,,,,,,,,,,,,
2012-01-03,0.043537,0.030814,0.034304,0.024671,0.020945,0.017285,-0.011462,0.029801,0.023306,0.0,0.029193,-0.004539
2012-01-04,0.006765,0.007114,0.006225,0.001146,0.001593,0.017983,-0.023517,0.018820,0.005222,0.0,0.001742,-0.004975
2012-01-05,-0.027102,-0.011587,-0.006225,-0.007473,-0.006388,-0.010951,-0.019997,0.016807,0.006231,0.0,-0.004361,-0.010445
2012-01-06,-0.008637,0.000896,-0.000833,0.007473,0.012419,0.014903,-0.002528,0.011599,-0.011453,0.0,0.005231,-0.025522
2012-01-09,-0.001206,0.013345,0.019803,0.006280,0.002528,0.015656,-0.001949,-0.033504,0.000000,0.0,0.006932,0.008153


## Buy & Hold

In [81]:
#std buy-and-hold train and test
Ytrain.std(), Ytest.std()

(0.014595554557542332, 0.01811346510486715)

In [82]:
#SR buy-and-hold train and test
Ytrain.mean()/Ytrain.std(), Ytest.mean()/Ytest.std()

(0.011830254737761498, 0.01884965966243372)

In [83]:
# Total return buy-and-hold train and test
Ytrain.sum(), Ytest.sum()

(0.25814034703965927, 0.3410912198815809)

## XGBoost Regressor

In [84]:
from xgboost.sklearn import XGBRegressor
model = XGBRegressor()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.9754967674930669, -0.33975278393401354)

In [85]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.9351170568561873, 0.4974974974974975)

In [86]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [87]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [88]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [89]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['BOVA11.SA']

In [90]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

8.193485808643503

In [91]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.26749686703545184

In [92]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.008873423254762225, 0.6176412819259426)

In [93]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.013408548105474824, 0.019969696164030496)

In [94]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(5.216437245078913e-06, 0.00043912968755164855)

In [95]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.002283952110942546, 0.020955421435791946)

In [96]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.0015250622983159366, 0.014147395639639626)

In [97]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(0.9754967674930669, -0.33975278393401354)

In [98]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(113375202202.2522, 64242548968.50831)

In [99]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

0.33815518366321834

In [100]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.4821197122831042

In [101]:
df_returns['Position'].to_csv('XGBRBool.csv', index = True)
sum(df_returns['Position'])

1238

## LGBM Regressor

In [102]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.7984442605322056, -0.11833021419685186)

In [103]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.8561872909698997, 0.4824824824824825)

In [104]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [105]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [106]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [107]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['BOVA11.SA']

In [108]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

7.768042705590982

In [109]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

-0.20618508661885837

In [110]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.009076243809117738, 0.5724851928797359)

In [111]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.01515243111112356, -0.013621014118681007)

In [112]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(4.2908741367969805e-05, 0.0003665541907648086)

In [113]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.006550476422976409, 0.019145604998662448)

In [114]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.0048980466216210295, 0.012831085039564616)

In [115]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(0.7984442605322056, -0.11833021419685186)

In [116]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(293918420570.1888, 30806525617.649574)

In [117]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

0.7787665101024225

In [118]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.5264196957178189

In [119]:
df_returns['Position'].to_csv('LGBMRBool.csv', index = True)
sum(df_returns['Position'])

1317

In [120]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score

## XGBoost Classifier

In [121]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

C:\Users\rapha\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:36:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.9979933110367893, 0.5115115115115115)

In [122]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['BOVA11.SA']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [123]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

8.249759159316792

In [124]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.35779729352519934

In [125]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.008852812348104557, 0.623331133022825)

In [126]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.013366762843565656, 0.026794479199320757)

In [127]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.9979933110367893, 0.5115115115115115)

In [128]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.9959946595460614, 0.5354330708661418)

In [129]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.9979933110367892, 0.5271317829457365)

In [130]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.9979973297730307, 0.5111209321012454)

In [131]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.49017979491118485, 0.5081200833788774)

In [132]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7001284131580326, 0.7128254227921991)

In [133]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5008306753531507, 0.5138989645235252)

In [134]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-2301.5273612254227, -1549.2374710194617)

In [135]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(162.7301970221022, 18032430940589.723)

In [136]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.977596807899588

In [137]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9871206457174806

In [138]:
df_returns['Position'].to_csv('XGBCBool.csv', index = True)
sum(df_returns['Position'])

1257

## LGBM Classifier

In [139]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.9979933110367893, 0.5115115115115115)

In [140]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['BOVA11.SA']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [141]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

8.249759159316792

In [142]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.22964037162149342

In [143]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.008852812348104557, 0.623331133022825)

In [144]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.012763970319667736, 0.01800930557705501)

In [145]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.9979933110367893, 0.5115115115115115)

In [146]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.9959946595460614, 0.5384615384615384)

In [147]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.9979933110367892, 0.5080645161290324)

In [148]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.9979973297730307, 0.513089594214544)

In [149]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.49017979491118485, 0.4683366137530591)

In [150]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7001284131580326, 0.6843512356626962)

In [151]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5008306753531507, 0.47450656408576614)

In [152]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-2301.5273612254227, -1427.860994555489)

In [153]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(162.7301970221022, 9016215470392.55)

In [154]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.977596807899588

In [155]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.98835997700613

In [156]:
df_returns['Position'].to_csv('LGBMCBool.csv', index = True)
sum(df_returns['Position'])

1217

## KNN

In [157]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.6996655518394649, 0.5445445445445446)

In [158]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['BOVA11.SA']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [159]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

3.425973711426373

In [160]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.8850797877417707

In [161]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.009926402462153218, 0.23086120277651345)

In [162]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.011691737095646547, 0.07577708481190172)

In [163]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.6996655518394649, 0.5445445445445446)

In [164]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.7088607594936709, 0.5726315789473684)

In [165]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.6918325326012353, 0.5445445445445445)

In [166]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.6996173629182073, 0.5458577742065086)

In [167]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.4712149593899681, 0.47403142973680223)

In [168]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.6864509883378187, 0.6884994043111455)

In [169]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.47898443049087225, 0.4810744187163992)

In [170]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-2212.4436145225354, -1445.2354645230073)

In [171]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(30124412223356.676, 4508107735273.629)

In [172]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.9849379267542953

In [173]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9874778205277726

In [174]:
df_returns['Position'].to_csv('KNNBool.csv', index = True)
sum(df_returns['Position'])

1186

## Naive Bayes Gaussian Classifier

In [175]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.5337792642140469, 0.4944944944944945)

In [176]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['BOVA11.SA']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [177]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.5476056785565273

In [178]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.5221636546963317

In [179]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.009545528468686362, 0.038373090079321576)

In [180]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.00590626811943088, 0.088496886776575)

In [181]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.5337792642140469, 0.4944944944944945)

In [182]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.5463137996219282, 0.611764705882353)

In [183]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.45333333333333325, 0.17077175697865352)

In [184]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.5334861137459418, 0.5148814785054239)

In [185]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.35332648853093107, 0.08436759817132765)

In [186]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.5944127257478015, 0.2904610097264823)

In [187]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.360173971841477, 0.09529894719233144)

In [188]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-1658.6847029069702, -256.3994146332183)

In [189]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(39161735890275.555, 20.324165356567992)

In [190]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.990872678187865

In [191]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9965549450880964

In [192]:
df_returns['Position'].to_csv('NBGCBool.csv', index = True)
sum(df_returns['Position'])

614